In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
esg = pd.read_csv('SP 500 ESG Risk Ratings.csv')
gdp = pd.read_excel('gdp_raw.xlsx')
state_name = pd.read_excel('state_name_raw.xlsx')
fin = pd.read_excel('fin_raw.xlsx')

# 1. Data Cleaning for ESG dataset

In [3]:
# have a basic understanding of the whole dataset
esg.describe()

,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Score
count,433.000000,433.000000,433.000000,433.000000,433.000000
mean,21.422633,5.678984,6.673903,9.045035,1.896074
std,7.264458,5.339965,2.201049,3.813788,0.940748
min,7.000000,0.000000,3.000000,1.100000,0.000000
25%,16.000000,1.500000,5.000000,6.600000,1.000000
50%,21.000000,3.800000,6.000000,8.700000,2.000000
75%,26.000000,8.900000,7.700000,11.600000,2.000000
max,46.000000,25.000000,15.500000,21.000000,5.000000


In [4]:
# check the data type
esg.dtypes

Symbol                     object
Name                       object
Address                    object
Sector                     object
Industry                   object
Full Time Employees        object
Description                object
Total ESG Risk score      float64
Environment Risk Score    float64
Governance Risk Score     float64
Social Risk Score         float64
Controversy Level          object
Controversy Score         float64
ESG Risk Percentile        object
ESG Risk Level             object
dtype: object

In [5]:
# change the type of "Full Time Employees" from object to int
esg['Full Time Employees'] = esg['Full Time Employees'].apply(lambda x: x.replace(',', '').strip() if pd.notnull(x) else '0')
esg['Full Time Employees'] = esg['Full Time Employees'].astype(int)

In [6]:
# check if esg_data has null value
null_counts = esg.isnull().sum()
null_counts

Symbol                      0
Name                        0
Address                     2
Sector                      2
Industry                    2
Full Time Employees         0
Description                 2
Total ESG Risk score       70
Environment Risk Score     70
Governance Risk Score      70
Social Risk Score          70
Controversy Level          99
Controversy Score          70
ESG Risk Percentile        70
ESG Risk Level            127
dtype: int64

In [7]:
# test 1: the relationship between "missing Total ESG Risk score" and "missing detailed ESG risk scores"
null_indexes_1 = esg[esg['Total ESG Risk score'].isnull() & 
                     esg['Environment Risk Score'].isnull() & 
                     esg['Governance Risk Score'].isnull()].index

In [8]:
# conclusion: if the company doesn't have Total ESG Risk score, the other 3 scores will be empty
# should delete 70 record later
esg.iloc[null_indexes_1]

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Description,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Percentile,ESG Risk Level
23,ALGN,Align Technology Inc,"410 North Scottsdale Road\nSuite 1300\nTempe, ...",Healthcare,Medical Devices,22910,"Align Technology, Inc. designs, manufactures, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,ALK,Alaska Air Group Inc,"19300 International Boulevard\nSeattle, WA 981...",Industrials,Airlines,23140,"Alaska Air Group, Inc., through its subsidiari...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,AMCR,Amcor Plc,Thurgauerstrasse 34\nZurich 8050\nSwitzerland\...,Consumer Cyclical,Packaging & Containers,41000,"Amcor plc develops, produces, and sells packag...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,AMD,Advanced Micro Devices,"2485 Augustine Drive\nSanta Clara, CA 95054\nU...",Technology,Semiconductors,25000,"Advanced Micro Devices, Inc. operates as a sem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,AON,Aon Plc Class A,Metropolitan Building\nJames Joyce Street\nDub...,Financial Services,Insurance Brokers,50000,"Aon plc, a professional services firm, provide...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,WBD,Warner Bros Discovery Inc,"230 Park Avenue South\nNew York, NY 10003\nUni...",Communication Services,Entertainment,37500,"Warner Bros. Discovery, Inc., operates a media...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
490,WRK,Westrock Co,"1000 Abernathy Road NE\nAtlanta, GA 30328\nUni...",Consumer Cyclical,Packaging & Containers,50500,"WestRock Company, together with its subsidiari...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,WST,West Pharmaceutical Services,"530 Herman O. West Drive\nExton, PA 19341-0645...",Healthcare,Medical Instruments & Supplies,10700,"West Pharmaceutical Services, Inc. designs, ma...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,ZBRA,Zebra Technologies Corp Cl A,"3 Overlook Point\nLincolnshire, IL 60069\nUnit...",Technology,Communication Equipment,10500,"Zebra Technologies Corporation, together with ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# test 2: the relationship between "missing Total ESG Risk score" and "missing location, industry information"
null_indexes_2 = esg[esg['Address'].isnull() & 
                     esg['Sector'].isnull() & 
                     esg['Industry'].isnull()].index

In [10]:
# conclusion: only 1 company with ESG score but without company's location info
# delete 1 record
esg.iloc[null_indexes_2]

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Description,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Percentile,ESG Risk Level
61,BF.B,Brown Forman Corp Class B,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,CAT,Caterpillar Inc,NaN,NaN,NaN,0,NaN,34.0,7.2,8.5,18.5,High,4.0,76th percentile,High


In [11]:
# drop the records based on the conclusions above
null_indexes_combined = esg[
    (
        (esg['Total ESG Risk score'].isnull() &
        esg['Environment Risk Score'].isnull() &
        esg['Governance Risk Score'].isnull() &
        esg['Social Risk Score'].isnull())
    ) |
    (
        (esg['Address'].isnull() &
        esg['Sector'].isnull() &
        esg['Industry'].isnull())
    )
].index

esg.drop(null_indexes_combined, inplace=True)
esg.reset_index(drop=True, inplace=True)

In [12]:
# calculate the null values again to see if we have dropped all records we want
# won't analyze the other cols so leave the null values there
esg.isnull().sum()

Symbol                     0
Name                       0
Address                    0
Sector                     0
Industry                   0
Full Time Employees        0
Description                0
Total ESG Risk score       0
Environment Risk Score     0
Governance Risk Score      0
Social Risk Score          0
Controversy Level         29
Controversy Score          0
ESG Risk Percentile        0
ESG Risk Level            57
dtype: int64

In [13]:
# no repeated value in the dataset
len(esg['Name']) == len(esg['Name'].unique())

True

In [14]:
esg

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Description,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Percentile,ESG Risk Level
0,A,Agilent Technologies Inc,"5301 Stevens Creek Boulevard\nSanta Clara, CA ...",Healthcare,Diagnostics & Research,18000,"Agilent Technologies, Inc. provides applicatio...",15.0,0.3,6.3,8.6,Low,1.0,11th percentile,Low
1,AAL,American Airlines Group Inc,"1 Skyview Drive\nFort Worth, TX 76155\nUnited ...",Industrials,Airlines,132500,"American Airlines Group Inc., through its subs...",29.0,12.0,5.0,12.0,Moderate,2.0,62nd percentile,NaN
2,AAP,Advance Auto Parts Inc,"4200 Six Forks Road\nRaleigh, NC 27609\nUnited...",Consumer Cyclical,Specialty Retail,40000,"Advance Auto Parts, Inc. provides automotive r...",12.0,0.0,3.0,8.0,Moderate,2.0,4th percentile,Negligible
3,AAPL,Apple Inc,"One Apple Park Way\nCupertino, CA 95014\nUnite...",Technology,Consumer Electronics,164000,"Apple Inc. designs, manufactures, and markets ...",17.0,0.6,9.2,6.9,Significant,3.0,15th percentile,Low
4,ABBV,Abbvie Inc,"1 North Waukegan Road\nNorth Chicago, IL 60064...",Healthcare,Drug Manufacturers—General,50000,"AbbVie Inc. discovers, develops, manufactures,...",28.0,1.1,9.9,16.8,Significant,3.0,55th percentile,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,XRAY,Dentsply Sirona Inc,"13320 Ballantyne Corporate Place\nCharlotte, N...",Healthcare,Medical Instruments & Supplies,15000,DENTSPLY SIRONA Inc. manufactures and sells va...,16.0,2.0,7.0,7.0,Low,1.0,12th percentile,Negligible
428,XYL,Xylem Inc,"300 Water Street SE\nWashington, DC 20003\nUni...",Industrials,Specialty Industrial Machinery,22000,"Xylem Inc., together with its subsidiaries, en...",16.0,4.0,5.0,7.0,Low,1.0,13th percentile,Negligible
429,YUM,Yum Brands Inc,"1441 Gardiner Lane\nLouisville, KY 40213\nUnit...",Consumer Cyclical,Restaurants,36000,"Yum! Brands, Inc., together with its subsidiar...",21.0,4.5,4.4,11.6,Moderate,2.0,27th percentile,Medium
430,ZBH,Zimmer Biomet Holdings Inc,"345 East Main Street\nWarsaw, IN 46580\nUnited...",Healthcare,Medical Devices,18000,"Zimmer Biomet Holdings, Inc., together with it...",27.0,4.0,8.0,15.0,Moderate,2.0,54th percentile,NaN


In [238]:
# already stored under "processed" folder
#esg.to_excel('data/processed/esg_clean.xlsx', index=False)

# 2. Data Cleaning for GDP dataset

In [15]:
gdp

,state,gdp_2022,gdp_2023
0,2022,2023,2022
1,NaN,NaN,NaN
2,California *,"3,598,103","3,755,487"
3,Texas *,"2,355,960","2,436,346"
4,New York *,"2,053,180","2,135,672"
5,Florida *,"1,389,070","1,468,015"
6,Illinois *,"1,033,310","1,071,552"
7,Pennsylvania *,"923,089","961,946"
8,Ohio *,"822,670","852,903"
9,Georgia *,"755,698","792,151"


In [16]:
# check the detail of "state" column before cleaning
gdp['state'].tolist()

['2022',
 nan,
 'California\u202f*',
 'Texas\u202f*',
 'New York\u202f*',
 'Florida\u202f*',
 'Illinois\u202f*',
 'Pennsylvania\u202f*',
 'Ohio\u202f*',
 'Georgia\u202f*',
 'New Jersey\u202f*',
 'North Carolina\u202f*',
 'Washington\u202f*',
 'Massachusetts\u202f*',
 'Virginia\u202f*',
 'Michigan\u202f*',
 'Colorado\u202f*',
 'Tennessee\u202f*',
 'Maryland\u202f*',
 'Arizona\u202f*',
 'Indiana\u202f*',
 'Minnesota\u202f*',
 'Wisconsin\u202f*',
 'Missouri\u202f*',
 'Connecticut\u202f*',
 'Oregon\u202f*',
 'South Carolina\u202f*',
 'Louisiana\u202f*',
 'Alabama\u202f*',
 'Kentucky\u202f*',
 'Utah\u202f*',
 'Oklahoma\u202f*',
 'Iowa\u202f*',
 'Nevada\u202f*',
 'Kansas\u202f*',
 'Arkansas\u202f*',
 'Nebraska\u202f*',
 'District of Columbia*',
 'Mississippi\u202f*',
 'New Mexico\u202f*',
 'Idaho\u202f*',
 'New Hampshire\u202f*',
 'Hawaii\u202f*',
 'West Virginia\u202f*',
 'Delaware\u202f*',
 'Maine\u202f*',
 'North Dakota\u202f*',
 'Rhode Island\u202f*',
 'South Dakota\u202f*',
 'Montana\u2

In [17]:
# drop irrelevant rows
gdp.drop([0,1,53],inplace=True)
gdp.reset_index(drop=True, inplace=True)

In [18]:
# clean state column
gdp['state'] = gdp['state'].apply(lambda x: re.sub(r'[\u202f*]', '', x))

In [19]:
# clean gdp columns, change data type from str to int
gdp['gdp_2022'] = gdp['gdp_2022'].apply(lambda x: x.replace(',', '').strip()).astype(int)
gdp['gdp_2023'] = gdp['gdp_2023'].apply(lambda x: x.replace(',', '').strip()).astype(int)

In [20]:
# after cleaning
gdp['state'].tolist()

['California',
 'Texas',
 'New York',
 'Florida',
 'Illinois',
 'Pennsylvania',
 'Ohio',
 'Georgia',
 'New Jersey',
 'North Carolina',
 'Washington',
 'Massachusetts',
 'Virginia',
 'Michigan',
 'Colorado',
 'Tennessee',
 'Maryland',
 'Arizona',
 'Indiana',
 'Minnesota',
 'Wisconsin',
 'Missouri',
 'Connecticut',
 'Oregon',
 'South Carolina',
 'Louisiana',
 'Alabama',
 'Kentucky',
 'Utah',
 'Oklahoma',
 'Iowa',
 'Nevada',
 'Kansas',
 'Arkansas',
 'Nebraska',
 'District of Columbia',
 'Mississippi',
 'New Mexico',
 'Idaho',
 'New Hampshire',
 'Hawaii',
 'West Virginia',
 'Delaware',
 'Maine',
 'North Dakota',
 'Rhode Island',
 'South Dakota',
 'Montana',
 'Alaska',
 'Wyoming',
 'Vermont']

In [21]:
# add one more column as "change_percentage" to calculate the percentage change of gdp from 2022 to 2023
gdp['change_percentage'] = (gdp['gdp_2023'] - gdp['gdp_2022'])/gdp['gdp_2022']

In [22]:
gdp

,state,gdp_2022,gdp_2023,change_percentage
0,California,3598103,3755487,0.043741
1,Texas,2355960,2436346,0.034120
2,New York,2053180,2135672,0.040178
3,Florida,1389070,1468015,0.056833
4,Illinois,1033310,1071552,0.037009
5,Pennsylvania,923089,961946,0.042095
6,Ohio,822670,852903,0.036750
7,Georgia,755698,792151,0.048238
8,New Jersey,745422,776923,0.042259
9,North Carolina,730072,762866,0.044919


In [251]:
# already stored under "processed" folder 
#gdp.to_excel('data/processed/gdp_clean.xlsx', index=False)

# 3. Data Cleaning for State Name Dataset

In [23]:
state_name.head(5)

,state,state_abbreviation
0,United States of America,USUSA840
1,Alabama,US-AL
2,Alaska,US-AK
3,Arizona,US-AZ
4,Arkansas,US-AR


In [24]:
# have some non-match columns
print(len(state_name))
print(len(gdp))

78
51


In [25]:
state_name_gdp = pd.merge(state_name,gdp,on='state')

state_name_gdp['state_abbreviation'].fillna('Unknown', inplace=True)
state_name_gdp['state_abbreviation'] = state_name_gdp['state_abbreviation'].apply(lambda x: x.replace('US-',''))

In [26]:
state_name_gdp.head(5)

,state,state_abbreviation,gdp_2022,gdp_2023,change_percentage
0,Alabama,AL,277817,289038,0.040390
1,Alaska,AK,63618,63750,0.002075
2,Arizona,AZ,458950,479759,0.045340
3,Arkansas,AR,165221,171152,0.035897
4,California,CA,3598103,3755487,0.043741


In [256]:
# already stored under "processed" folder
#state_name_gdp.to_excel('data/processed/state_name_gdp_clean.xlsx', index=False)

# 4. Data Cleaning for Yahoo Finance Dataset

In [27]:
fin.rename(columns={'symbol': 'Symbol'}, inplace=True)

In [28]:
fin

,Symbol,market_value,latest_revenue,stock_volatility
0,A,33349068800,6848000000,0.016883
1,AAL,8103907840,48971000000,0.023060
2,AAP,3153115392,11154722000,0.031097
3,AAPL,2977583333376,383285000000,0.014296
4,ABBV,244191821824,58054000000,0.012325
...,...,...,...,...
498,YUM,35913060352,6842000000,0.010160
499,ZBH,23510362112,6939900000,0.013304
500,ZBRA,11326845952,5781000000,0.025469
501,ZION,5300770816,3152000000,0.039206


In [29]:
# check null value
fin.isnull().sum()

Symbol              0
market_value        0
latest_revenue      0
stock_volatility    2
dtype: int64

In [30]:
# delete BF.B, cuz this company doesn't have ESG scores, also deleted in the previous step
# keep company ABC
fin[fin['stock_volatility'].isnull()]

,Symbol,market_value,latest_revenue,stock_volatility
5,ABC,0,0,NaN
61,BF.B,0,0,NaN


In [31]:
esg[esg['Symbol']=='ABC']

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Description,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Percentile,ESG Risk Level
5,ABC,Amerisourcebergen Corp,"1 West First Avenue\nConshohocken, PA 19428-18...",Healthcare,Medical Distribution,46000,AmerisourceBergen Corporation sources and dist...,12.0,1.3,5.2,5.6,Significant,3.0,5th percentile,Low


In [32]:
fin.drop([61],inplace=True)
fin.reset_index(drop=True, inplace=True)

In [33]:
fin

,Symbol,market_value,latest_revenue,stock_volatility
0,A,33349068800,6848000000,0.016883
1,AAL,8103907840,48971000000,0.023060
2,AAP,3153115392,11154722000,0.031097
3,AAPL,2977583333376,383285000000,0.014296
4,ABBV,244191821824,58054000000,0.012325
...,...,...,...,...
497,YUM,35913060352,6842000000,0.010160
498,ZBH,23510362112,6939900000,0.013304
499,ZBRA,11326845952,5781000000,0.025469
500,ZION,5300770816,3152000000,0.039206


In [264]:
# already stored under "processed" folder
#fin.to_excel('data/processed/fin_clean.xlsx', index=False)